# DOMAIN: Smartphone, Electronics
• CONTEXT: India is the second largest market globally for smartphones after China. About 134 million smartphones were sold across India in the year 2017 and is estimated to increase to about 442 million in 2022. India ranked second in the average time spent on mobile web by smartphone users across Asia Pacific. The combination of very high sales volumes and the average smartphone consumer behaviour has made India a very attractive market for foreign vendors. As per Consumer behaviour, 97% of consumers turn to a search engine when they are buying a product vs. 15% who turn to social media. If a seller succeeds to publish smartphones based on user’s behaviour/choice at the right place, there are 90% chances that user will enquire for the same. This Case Study is targeted to build a recommendation system based on individual consumer’s behaviour or choice.

• DATA DESCRIPTION:

• author : name of the person who gave the rating

• country : country the person who gave the rating belongs to

• data : date of the rating

• domain: website from which the rating was taken from

• extract: rating content

• language: language in which the rating was given

• product: name of the product/mobile phone for which the rating was given

• score: average rating for the phone

• score_max: highest rating given for the phone

• source: source from where the rating was taken

• PROJECT OBJECTIVE: We will build a recommendation system using popularity based and collaborative filtering methods to recommend mobile phones to a user which are most popular and personalised respectively

# 1. Import the necessary libraries and read the provided CSVs as a data frame and perform the below steps. [15 Marks]

In [1]:
#need surprise library to be installed
#The name SurPRISE (roughly :) ) stands for Simple Python RecommendatIon System Engine.
#Surprise is a Python scikit for building and analyzing recommender systems that deal with explicit rating data.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from surprise import Dataset,Reader
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import SVD, KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import os
import warnings
#warnings.filterwarnings()

In [3]:
!pip install scikit-surprise 


# 1A. Merge all the provided CSVs into one dataFrame. [2 Marks]

In [4]:
df1=pd.read_csv('phone_user_review_file_1.csv',encoding='latin1')
df2=pd.read_csv('phone_user_review_file_2.csv',encoding='latin1')
df3=pd.read_csv('phone_user_review_file_3.csv',encoding='latin1')
df4=pd.read_csv('phone_user_review_file_4.csv',encoding='latin1')
df5=pd.read_csv('phone_user_review_file_5.csv',encoding='latin1')
df6=pd.read_csv('phone_user_review_file_6.csv',encoding='latin1')
#latin1 encoding to avoid unicodeDecode error

In [5]:
df=df1.append([df2,df3,df4,df5,df6]) #since we have same columns we are appending the each dataframe in order to merge all the provided CSVs into one dataframe

C:\Users\91984\AppData\Local\Temp\ipykernel_10404\4186925817.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append([df2,df3,df4,df5,df6]) #since we have same columns we are appending the each dataframe in order to merge all the provided CSVs into one dataframe


# 1B. Explore, understand the Data and share at least 2 observations. [2 Marks]

In [6]:
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [7]:
# Shape OF data

df.shape

(1415133, 11)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


Insight: We have 1415133 Rows and 11 Columns. 

Dataset contains some missing values in columns like score,score_max,extract,author,product where non-null values are not equal to number of rows i.e 1415133 and contains object datatypes as well.


# 2C. Round off scores to the nearest integers. [3 Marks]

In [9]:
df['score']=df['score'].apply(np.round) #to round off score to the nearest integer

In [10]:
# Average of score i.e ratings given by all users to all products
np.round(df['score'].mean())

8.0

In [11]:
# Since date column is object Converting it into datetime datatype and also creating year and Month Column for further analysis

df['date']=pd.to_datetime(df['date'])
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

C:\Users\91984\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17/11/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\91984\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '29/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\91984\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '27/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\91984\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26/10/2015' in DD/MM/YYYY format. Provide format or specif

# 2D. Check for missing values. Impute the missing values, if any. [2 Marks]

In [12]:
df.isnull().sum() #to check the missing/null values

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
year             0
month            0
dtype: int64

Insight: score , score_max, extract, author and product has missing values.

Missing values can be imputed using mean/mode.

Imputing score with mean value of column 'score' and rest of the columns which are of object datatype with mode.

# Missing Value Imputation

In [13]:
df['author'].fillna(df['author'].mode()[0],inplace=True)
df['extract'].fillna(df['extract'].mode()[0],inplace=True)
df['score'].fillna(np.round(df['score'].mean()),inplace=True)
df['product'].fillna(df['product'].mode()[0],inplace=True)

# 1E. Check for duplicate values and remove them, if any. [2 Marks]

In [14]:
df.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score            0
score_max    63489
extract          0
author           0
product          0
year             0
month            0
dtype: int64

In [15]:
df['score_max'].value_counts()

10.0    1351644
Name: score_max, dtype: int64

In [16]:
#As The column 'Score_max' is having unique/same value for all the rows , the same can be dropped.
df.drop('score_max',axis=1,inplace=True)

In [17]:
df.isnull().sum()

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
extract      0
author       0
product      0
year         0
month        0
dtype: int64

In [18]:
df.drop_duplicates(inplace=True) #to drop duplicates

In [19]:
df.shape

(1405202, 12)

Duplicate values are removed and the data is cleaned.

# 1F. Keep only 1 Million data samples. Use random state=612. [2 Marks]

In [20]:
df_sample=df.sample(n=100000,random_state=612) #to fetch 1 million data samples at random state 612.

In [21]:
df_sample.shape

(100000, 12)

# 1G. Drop irrelevant features. Keep features like Author, Product, and Score. [2 Marks]

In [22]:
df_sample.drop(['extract','domain','phone_url','date','lang','country','source','domain','year','month'],axis=1,inplace=True)
# Dropping the irrelavant features/columns which does not contribute significatntly for further analysis and Recommendation system Model

In [23]:
df_sample.shape

(100000, 3)

Now shape of the data is 1 Million rows with 3 coloumns

# 2A. Identify the most rated products. [3 Marks]

In [24]:
df_sample['product'].value_counts()[:10] 
#fetches the number of rows each product appeared in the data (i.e rated) in descending order so as the top one is most rated here

Lenovo Vibe K4 Note (White,16GB)       344
Lenovo Vibe K4 Note (Black, 16GB)      288
OnePlus 3 (Graphite, 64 GB)            281
OnePlus 3 (Soft Gold, 64 GB)           250
Samsung Galaxy Express I8730           216
Huawei P8lite zwart / 16 GB            185
Lenovo Vibe K5 (Gold, VoLTE update)    173
Nokia 5800 XpressMusic                 162
Samsung Galaxy S6 zwart / 32 GB        157
Lenovo Vibe K5 (Grey, VoLTE update)    148
Name: product, dtype: int64

Insight: Lenovo Vibe K4 Note (White) is the most rated product followed by Lenovo vibe k4 Note(black) and Oneplus 3 (Graphite)

# 2B. Identify the users with most number of reviews. [3 Marks]

In [25]:
df_sample['author'].value_counts()[:10] 
#fetches the number of rows each user appeared in the data (i.e who reviewed/rated) in descending order so as the top one is most reviewed user here

Amazon Customer    9952
Cliente Amazon     1405
e-bit               609
Client d'Amazon     550
Amazon Kunde        324
einer Kundin        205
Anonymous           178
unknown             144
einem Kunden        140
Anonymous           105
Name: author, dtype: int64

Insight:"Amazon Customer" has given higest number of reviews followed by Cliente Amazon

# 2C. Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the finaldataset. [4 Marks]

In [26]:
# selecting products which had atleast 50 reviews.

countsp=pd.value_counts(df_sample['product'])

df_sample=df_sample[ df_sample['product'].isin(countsp[countsp>50].index)]

In [27]:
#checking minimum product count
df_sample['product'].value_counts(ascending=True)[:5] 

HTC One V                                                       51
Nokia E72                                                       51
Apple iPhone 5 A1428 Factory Unlocked Cellphone, 16GB, Black    51
Nokia 3210                                                      52
Lenovo Vibe K5 Plus (Dark Grey, 16 GB)                          52
Name: product, dtype: int64

In [28]:
#selecting user count greater than 50
counts=pd.value_counts(df_sample['author']) 

In [29]:
df_sample=df_sample[(df_sample['author'].isin(counts[counts>50].index)) ]

In [30]:
df_sample['author'].value_counts(ascending=True)[:5] #checking minimum user count

Anonymous            51
einem Kunden         61
einer Kundin         76
Cliente Amazon       92
Amazon Customer    2386
Name: author, dtype: int64

In [31]:
df_sample.shape #Final data shape

(2666, 3)

# 3. Build a popularity based model and recommend top 5 mobile phones. [5 Marks]

As the name implies  Popularity based recommendation  system  works  with  the ongoing  trend.
    
It basically uses the items(smart phone here) which are more popular  at present.
    
The item that is more popular among the general public,  is more likely to be recommended to new customers. 
    
But this is more generalized recommendation rather than personalized

In [32]:
df_sample.groupby('product')['score'].mean().sort_values(ascending=False)[:5] # Top 5 mobile phones

product
Nokia 5800                                                                                                                                                   10.0
Samsung Galaxy S7 32GB (Verizon)                                                                                                                             10.0
Samsung Galaxy S7 32GB (AT&T)                                                                                                                                10.0
Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]    10.0
Samsung Galaxy S6 32GB (AT&T)                                                                                                                                10.0
Name: score, dtype: float64

# Insight:

Nokia 5800  is the most popular rated product with the score of 10.0 followed by Samsung Galaxy S7 32 GB(Verizon),AT&T,Lenovo Motoroal and samsung Galaxy S6 32 GN
#The Top 5 score ranges between 9.321429 and 9.068966.

# 4.Build a collaborative filtering model using SVD. You can use SVD from surprise or build it from scratch.Build a collaborative filtering model using kNNWithMeans from surprise. You can try both user-based and item-based model. [10 Marks]

In [33]:
df_sample.head()

,score,author,product
271040,2.0,Amazon Customer,Nokia E52
71638,10.0,Amazon Customer,"OnePlus 3 (Soft Gold, 64 GB)"
34614,6.0,einem Kunden,"Samsung Galaxy S7 Smartphone, 12,9 cm (5,1 Zol..."
44707,8.0,Amazon Customer,"Apple iPhone 5s GSM Unlocked Cellphone, 16 GB,..."
67437,10.0,Cliente Amazon,"Huawei P9 Lite Smartphone, LTE, Display 5.2'' ..."


In [34]:
#Creating dataframe with only author,product and score column

df_col=df_sample[['author','product','score']]

df_col.head()

,author,product,score
271040,Amazon Customer,Nokia E52,2.0
71638,Amazon Customer,"OnePlus 3 (Soft Gold, 64 GB)",10.0
34614,einem Kunden,"Samsung Galaxy S7 Smartphone, 12,9 cm (5,1 Zol...",6.0
44707,Amazon Customer,"Apple iPhone 5s GSM Unlocked Cellphone, 16 GB,...",8.0
67437,Cliente Amazon,"Huawei P9 Lite Smartphone, LTE, Display 5.2'' ...",10.0


In [35]:
reader = Reader(rating_scale=(1, 10))

The Reader class from Surprise library is used to parse a file containing ratings.

Such a file is assumed to specify only one rating per line, and each line needs to respect the following structure:

user ; item ; rating ; [timestamp]

parameter:rating_scale (tuple, optional) – The rating scale used for every rating. Default is (1, 5).

In [36]:
#in surprise, there are three different data classes, each with their own distinct usage: 
#Dataset: used to split into train- and testsets, either directly, or through a cross-validation iterator. 

data= Dataset.load_from_df(df_col[['author','product','score']],reader)

Points to Note:

Trainset is no longer a pandas dataframe. Rather, it's a specific datatypes defined by the Surprise library.

Author and Product in the pandas dataframe can contain any value (either string/integer etc).

However, Trainset convert these raw ids into numeric indexes called as "inner id".

Methods are provided to convert raw id to inner id and vice versa

# Splitting data into train and test

we need to split our ‘ratings’ data frame into two parts — part 1 to train the algorithm to predict ratings and part 2 to test whether the rating predicted is close to what was expected. This will help in evaluating our models.

In [37]:
trainset, testset = train_test_split(data, test_size=.25,random_state=123)

# Building Collaborative Filtering Model using SVD.

In [38]:
svd_model = SVD(n_factors=5,biased=False)
svd_model.fit(trainset)

In [39]:
# Creating copy for future use maybe.
df_sam=df_col.copy()

Predicting values for Test data

In [40]:
test_pred_svd = svd_model.test(testset)

Compute RMSE

In [41]:
accuracy.rmse(test_pred_svd)

RMSE: 2.9791


2.979147302972237

# Building Collaborative Filtering Model using KNNWithMeans.

# User Based Model

In [42]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Predicting average rating for test data

In [43]:
# run the trained model against the testset
test_pred_user = algo.test(testset)

In [44]:
test_pred_user

[Prediction(uid='Amazon Customer', iid='YU Yuphoria YU5010A (Black+Silver)', r_ui=2.0, est=5.727272727272727, details={'actual_k': 44, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Motorola Moto G, 4th Gen (White, 2 GB, 16 GB)', r_ui=8.0, est=6.758620689655172, details={'actual_k': 29, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Nokia 5130 XpressMusic', r_ui=8.0, est=9.0, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Samsung Wave 525 GT-S5250', r_ui=6.0, est=5.999999999999999, details={'actual_k': 18, 'was_impossible': False}),
 Prediction(uid='Cliente Amazon', iid='Huawei P8 lite Smartphone, Display 5.0" IPS, Dual Sim, Processore Octa-Core, Memoria 16 GB, Fotocamera 13 MP, Android 5.0, Bianco', r_ui=10.0, est=8.133333333333333, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Lenovo Vibe K4 Note (Black, 16GB)', r_ui=6.0, est=6.56, details={'actual_k': 50

Evaluation-RMSE

In [45]:
# getting RMSE
print("User-based Model : Test Set")
accuracy.rmse(test_pred_user, verbose=True)

User-based Model : Test Set
RMSE: 2.7709


2.7709404163728526

Test Predictions for User Model

In [46]:
pred_df=pd.DataFrame(test_pred_user)

In [47]:
pred_df.head()

,uid,iid,r_ui,est,details
0,Amazon Customer,YU Yuphoria YU5010A (Black+Silver),2.0,5.727273,"{'actual_k': 44, 'was_impossible': False}"
1,Amazon Customer,"Motorola Moto G, 4th Gen (White, 2 GB, 16 GB)",8.0,6.758621,"{'actual_k': 29, 'was_impossible': False}"
2,Amazon Customer,Nokia 5130 XpressMusic,8.0,9.000000,"{'actual_k': 14, 'was_impossible': False}"
3,Amazon Customer,Samsung Wave 525 GT-S5250,6.0,6.000000,"{'actual_k': 18, 'was_impossible': False}"
4,Cliente Amazon,"Huawei P8 lite Smartphone, Display 5.0"" IPS, D...",10.0,8.133333,"{'actual_k': 15, 'was_impossible': False}"


# Steps in User based Collaborative filtering model.
Finding neighbours for the particular user based on the similarity measure.

50 neighbours are selected in this project.

Filter out the products which are already rated by the user

For remaining product find out the mean rating and sort the ratings in descending order and recommend to the user.

So user may get recommendation for the product which are highly rated by the user like him.

Also average rating for the product which is not rated by user can be estimated by its neighbours.

# Item Based Model

 For Item based model all the above steps are same except user_based is set to False

In [48]:
algo_item = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo_item.fit(trainset)  

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Predicting Average Rating for test data

# run the trained model against the testset


In [49]:
test_pred_item = algo_item.test(testset)

In [50]:
test_pred_item

[Prediction(uid='Amazon Customer', iid='YU Yuphoria YU5010A (Black+Silver)', r_ui=2.0, est=5.728119515725176, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Motorola Moto G, 4th Gen (White, 2 GB, 16 GB)', r_ui=8.0, est=6.758168928384057, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Nokia 5130 XpressMusic', r_ui=8.0, est=9.0, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Samsung Wave 525 GT-S5250', r_ui=6.0, est=5.978323870428543, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Cliente Amazon', iid='Huawei P8 lite Smartphone, Display 5.0" IPS, Dual Sim, Processore Octa-Core, Memoria 16 GB, Fotocamera 13 MP, Android 5.0, Bianco', r_ui=10.0, est=8.132138811511618, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Lenovo Vibe K4 Note (Black, 16GB)', r_ui=6.0, est=6.56, details={'actual_k': 50

# 5. Evaluate the collaborative model. Print RMSE value. [2 Marks]

Evaluation-RMSE

In [51]:
# getting RMSE
print("User-based Model : Test Set")
accuracy.rmse(test_pred_user, verbose=True)

User-based Model : Test Set
RMSE: 2.7709


2.7709404163728526

In [52]:
# getting RMSE
print("Item-based Model : Test Set")
accuracy.rmse(test_pred_item, verbose=True)

Item-based Model : Test Set
RMSE: 2.7684


2.76840610646908

Insight: Slightly better than User Based Model.

# 6. Predict score (average rating) for test users. [2 Marks]

# Test Predictions for Item Based Model

In [53]:
pred_df_item=pd.DataFrame(test_pred_item)

In [54]:
pred_df_item.head()

,uid,iid,r_ui,est,details
0,Amazon Customer,YU Yuphoria YU5010A (Black+Silver),2.0,5.728120,"{'actual_k': 50, 'was_impossible': False}"
1,Amazon Customer,"Motorola Moto G, 4th Gen (White, 2 GB, 16 GB)",8.0,6.758169,"{'actual_k': 50, 'was_impossible': False}"
2,Amazon Customer,Nokia 5130 XpressMusic,8.0,9.000000,"{'actual_k': 50, 'was_impossible': False}"
3,Amazon Customer,Samsung Wave 525 GT-S5250,6.0,5.978324,"{'actual_k': 50, 'was_impossible': False}"
4,Cliente Amazon,"Huawei P8 lite Smartphone, Display 5.0"" IPS, D...",10.0,8.132139,"{'actual_k': 50, 'was_impossible': False}"


# 7. Report your findings and inferences. [2 Marks]


# Insight:

RMSE for Item based model is higher than user based model.

SVD has less rmse value compared to others, may be change in paramters might provide better results.

# 8. Try and recommend top 5 products for test users. [5 Marks]

Recommending Top 5 products for the Unknown customers

In [55]:
df_sam['est']=df_sam['product'].apply(lambda x: svd_model.predict('avd',x).est)
df_sam = df_sam.sort_values(by=['est'], ascending=False)
df_sam.head(5)

,author,product,score,est
271040,Amazon Customer,Nokia E52,2.0,7.422211
241951,Amazon Customer,Nokia 2700 Classic,8.0,7.422211
359137,Amazon Customer,"Motorola Moto G, 4th Gen (Black, 2 GB, 16 GB)",2.0,7.422211
15129,einer Kundin,"Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,...",10.0,7.422211
367025,Amazon Customer,"OnePlus X (Onyx, 16GB)",10.0,7.422211


# 9. Try other techniques (Example: cross validation) to get better results.        [3 Marks]

# Cross Validation.

In [56]:
sim_options = {
    "name": ["msd", "cosine","pearson_baseline"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

In [57]:
param_grid = {'k':[40,50,60],"sim_options": sim_options}

Grid search CV to get best features.

In [58]:
gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=5)
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Best Parameters using Grid Search

In [59]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

2.8083852816946715
{'k': 60, 'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


Insight: Best Hyperparameters are: {'name': 'cosine', 'min_support': 3, 'user_based': False}

# Final Colloborative Model with Best Parameters.

In [60]:
Final_Model = KNNWithMeans(k=40, sim_options={'name': 'cosine', 'min_support':3 ,'user_based': False})
Final_Model.fit(trainset)
# run the trained model against the testset
Final_Model_Pred = algo_item.test(testset)
accuracy.rmse(Final_Model_Pred, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.7684


2.76840610646908

Insight: The Fianl RMSE score after applying BEst Hyperparametrs is : 2.7684

# Cross Validation on SVD Model.

In [65]:
param_grid = {'n_factors':[5,10],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [66]:
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

2.806162666886737
{'n_factors': 10, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


# Insight:

The Fianl RMSE score after applying BEst Hyperparametrs is : 2.01620
    
When Using cross validated best paramters SVD, RMSE score is less compared to others


# 10. In what business scenario you should use popularity based Recommendation Systems ? [2 Marks]

When we have new user who doesn't watch any of the movies we cant able to find its neighbours .

In this case popularity based recommendation is used.

As the name suggests Popularity based recommendation system works with the trend. 

It basically uses the items which are in trend right now.

For example, if any product which is usually bought by every new user then there are chances that it may suggest that item to the user who just signed up.

# 11. In what business scenario you should use CF based Recommendation Systems ? [2 Marks]

When user is not new user and he also got some neighbours. 

The idea behind collaborative filtering is to consider users’ opinions on different products and recommend the best product to each user based on the user’s previous rankings and the opinion of other similar types of users.

It has two types user-based and item-based.

Steps in user Based colloborative filtering:
    
1) Finding neighbours for the particular user based on the similarity measure.

2) 50 neighbours are selected in this project.

3) Filter out the movies which are already watched by the user

4) For remaining movies find out the mean rating and sort the movies in descending order and recommend to the user.

5) So user may get recommendation for the movie which are highly rated by the user like him.

6) Also average rating for the movie which is not watched by user can be estimated by its neighbours.

For item based every steps are based on items.

# 12. What other possible methods can you think of which can further improve the recommendation for different users ? [2 Marks]

In some cases we may face cold start or grey shep problem.In such cases, Combining any two filtering methods may help in improving the recommendation filter. These are known as Hybrid filtering.

In Hybrid filtering, which combines two or more filtering techniques in different ways in order to increase the accuracy and performance of recommender systems has been proposed. These techniques combine two or more filtering approaches in order to harness their strengths while leveling out their corresponding weaknesses.

When new users signs in, we may recommend using popularity based filtering and once users starts watching the movies then it can be switched to user based or item based . This can improve the recommendation.